# Project 6: IMDB

This project involves NLP, decision trees, bagging, boosting, and more!

---

## Load packages

You are likely going to need to install the `imdbpie` package:

    > pip install imdbpie

---

In [2]:
pip install imdbpie

SyntaxError: invalid syntax (<ipython-input-2-c1fc0cb5d435>, line 1)

In [2]:
import os
import subprocess
import collections
import re
import csv
import json

import pandas as pd
import numpy as np
import scipy

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

import psycopg2
import requests
from imdbpie import Imdb
import nltk

import urllib
from bs4 import BeautifulSoup
import nltk

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

---

## Part 1: Acquire the Data

You will connect to the IMDB API to query for movies. 

See here for documentation on how to use the package:

https://github.com/richardasaurus/imdb-pie

#### 1. Connect to the IMDB API

In [3]:
imdb = Imdb()
imdb = Imdb(anonymize=True)

In [ ]:
top250 = imdb.top_250()

#### 2. Query the top 250 rated movies in the database

In [11]:
top250

<bound method Imdb.top_250 of <imdbpie.imdbpie.Imdb object at 0x1141a5b90>>

AttributeError: 'Imdb' object has no attribute 'head'

#### 3. Make a dataframe from the movie data

Keep the fields:

    num_votes
    rating
    tconst
    title
    year
    
And discard the rest

In [5]:
def movie_parser(m):
    return [m['num_votes'], m['rating'], m['tconst'], m['title'], m['year']]

parsed = np.array([movie_parser(m) for m in top250])

movies = pd.DataFrame(parsed, columns = ['num_votes', 'rating', 'tconst', 'title', 'year'])

#### 3. Select only the top 100 movies

In [6]:
movies = movies[0:100]

#### 4. Get the genres and runtime for each movie and add them to the dataframe

There can be multiple genres per movie, so this will need some finessing.

In [8]:
runtimes = []
genres = []

for t in movies.tconst.values:
    title = imdb.get_title_by_id(t)
    print title.title
    runtimes.append(float(title.runtime))
    genres.append(title.genres)


The Shawshank Redemption
The Godfather
The Godfather: Part II
The Dark Knight
Schindler's List
12 Angry Men
Pulp Fiction
The Lord of the Rings: The Return of the King
The Good, the Bad and the Ugly
Fight Club
The Lord of the Rings: The Fellowship of the Ring
Star Wars: Episode V - The Empire Strikes Back
Forrest Gump
Inception
The Lord of the Rings: The Two Towers
One Flew Over the Cuckoo's Nest
Goodfellas
The Matrix
Seven Samurai
Star Wars: Episode IV - A New Hope
City of God
Se7en
The Silence of the Lambs
It's a Wonderful Life
The Usual Suspects
Life Is Beautiful
Léon: The Professional
Once Upon a Time in the West
Spirited Away
Saving Private Ryan
American History X
Interstellar
Casablanca
Psycho
City Lights
Raiders of the Lost Ark
The Intouchables
Rear Window
Modern Times
The Green Mile
Terminator 2: Judgment Day
The Pianist
The Departed
Back to the Future
Whiplash
Gladiator
Memento
Apocalypse Now
The Prestige
The Lion King
Dr. Strangelove or: How I Learned to Stop Worrying and Love

#### 4. Write the Results to a csv

In [9]:
print runtimes[0:4]
print genres[0:4]

[8520.0, 10500.0, 12120.0, 9120.0]
[[u'Crime', u'Drama'], [u'Crime', u'Drama'], [u'Crime', u'Drama'], [u'Action', u'Adventure', u'Crime', u'Thriller']]


In [10]:
flatten_genres = [
    item
    for sublist in genres
    for item in sublist
]

unique_genres = np.unique(flatten_genres)
print unique_genres

[u'Action' u'Adventure' u'Animation' u'Biography' u'Comedy' u'Crime'
 u'Drama' u'Family' u'Fantasy' u'Film-Noir' u'History' u'Horror' u'Music'
 u'Musical' u'Mystery' u'Romance' u'Sci-Fi' u'Thriller' u'War' u'Western']


In [11]:
genre_dummy_coded = []

for i, tconst in enumerate(movies.tconst.values):
    row = [tconst]
    row.extend([1 if g in genres[i] else 0 for ug in unique_genres])
    genre_dummy_coded.append(row)

NameError: name 'g' is not defined

In [12]:
movie_genres = pd.DataFrame(genre_dummy_coded, columns = ['tconst'] + unique_genres.tolist())
print movie_genres.head(3)
print movies.shape, movie_genres.shape

Empty DataFrame
Columns: [tconst, Action, Adventure, Animation, Biography, Comedy, Crime, Drama, Family, Fantasy, Film-Noir, History, Horror, Music, Musical, Mystery, Romance, Sci-Fi, Thriller, War, Western]
Index: []

[0 rows x 21 columns]
(100, 5) (0, 21)


In [14]:
movie = movies.merge(movie_genres, on = 'tcosnt')
print movies.shape
print movies.columns

KeyError: 'tcosnt'

In [ ]:
movies['runtimes'] = runtimes
movies.to_csv('./movies_with_genres.csv', index = False, encoding = 'utf-8')

---

## Part 2: Wrangle the text data

#### 1. Scrape the reviews for the top 100 movies

*Hint*: Use a loop to scrape each page at once

In [15]:
import requests

r = requests.get('http://www.imdb.com/title/tt0050083/reviews')

lines = r.text.split('\n')
lines[:10]

[u'',
 u'<!DOCTYPE html>',
 u'<html xmlns:og="http://opengraphprotocol.org/schema/"',
 u'      xmlns:fb="http://www.facebook.com/2008/fbml">',
 u'<head>',
 u'<meta http-equiv="content-type" content="text/html; charset=iso-8859-1">',
 u'<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime()};</script>',
 u'<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>',
 u'<script>',
 u'var addClickstreamHeadersToAjax = function(xhr) {']

In [16]:
def get_reviews(HTML):
    
    lines = HTML.split('\n')
    
    reviews = []
    review = []

# Chunk reviews into segments
    for lines in lines:
         
        if '<hr' in line:
            review.append(line)
            reviews.append(''.join(review))
            review = []
        else:
            review.append(line)
            
filtered_reviews = [review for review in reviews if 'people found the folling review useful' in review]

reviews = []

for review in filtered_reviews:
    
    rating = Selector(text = review).xpath("//img[@height='12']/@alt").extract()
    print rating
    
    if len(rating) > 0:
        split_rating = rating[0].split('/')
        rating = float(split_rating[0]) / float(split_rating[1])
    else:
        rating = None
    review = Selector(text=review).xpath('//p/text()').extract()[0]

NameError: name 'reviews' is not defined

#### 2. Extract the reviews and the rating per review for each movie

*Note*: "soup" from BeautifulSoup is the html returned from all 25 pages. You'll need to either address each page individually or break them down by elements.

#### 3. Remove the non AlphaNumeric characters from reviews

#### 4. Calculate the top 200 ngrams from the user reviews

Use the `TfidfVectorizer` in sklearn.

Recommended parameters:

    ngram_range = (1, 2)
    stop_words = 'english'
    binary = False
    max_features = 200

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer

tvc = TfidfVectorizer(ngram_range = (1, 2), stop_words = 'english')

#### 5. Merge the user reviews and ratings

#### 6. Save this merged dataframe as a csv

In [ ]:
movie_reviews.to_csv('reviews.csv', encoding='utf-8')

---

## Part 3: Combine Tables in PostgreSQL

#### 1. Import your two .csv data files into your Postgre Database as two different tables

For ease, we can call these table1 and table2

#### 2. Connect to database and query the joined set

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('postgres://localhost/movies2')

movie_column_mask = ['num_votes', 'rating', 'tconst', 'title', 'type', 'year', 'genres', 'runtime']

movie_review.to_sql('reviews', con = engine, if_exists='replace')

#### 3. Join the two tables 

In [ ]:
sql = """
SELECT m.movie, m.tconstr.rating, r.review FROM reviews r
LEFT JOIN movies m ON r.movie_id = m.tconst
LIMIT 10
"""

pd.read_sql(sql)

#### 4. Select the newly joined table and save two copies of the into dataframes

---

## Part 4: Parsing and Exploratory Data Analysis

#### 1. Rename any columns you think should be renamed for clarity

In [26]:
movies.rename({'tconst': 'movie_id'})

#### 2. Describe anything interesting or suspicious about your data (quality assurance)

#### 3. Make four visualizations of interest to you using the data

---

## Part 5: Decision Tree Classifiers and Regressors

#### 1. What is our target attribute? 

Choose a target variable for the decision tree regressor and the classifier. 

#### 2. Prepare the X and Y matrices and preprocess data as you see fit

In [19]:
movies = pd.read_csv('movies_user.csv')

In [20]:
movies.columns[0:50]

Index([u'num_votes', u'rating', u'tconst', u'title', u'year_x', u'Action',
       u'Adventure', u'Animation', u'Biography', u'Comedy', u'Crime', u'Drama',
       u'Family', u'Fantasy', u'Film-Noir', u'History', u'Horror', u'Music',
       u'Musical', u'Mystery', u'Romance', u'Sci-Fi', u'Thriller', u'War',
       u'Western', u'runtime', u'acting', u'action', u'actor', u'actors',
       u'actually', u'amazing', u'american', u'audience', u'away', u'bad',
       u'beautiful', u'believe', u'best', u'better', u'big', u'bit', u'black',
       u'book', u'brilliant', u'camera', u'cast', u'chaplin', u'character',
       u'characters'],
      dtype='object')

In [42]:
genres = movies.columns[5:25]
genres

Index([u'Action', u'Adventure', u'Animation', u'Biography', u'Comedy',
       u'Crime', u'Drama', u'Family', u'Fantasy', u'Film-Noir', u'History',
       u'Horror', u'Music', u'Musical', u'Mystery', u'Romance', u'Sci-Fi',
       u'Thriller', u'War', u'Western'],
      dtype='object')

In [63]:
duplicated_cols = ['rating', 'year_x', 'num_votes', 'runtime']
ngram2_col = [c for c in movies.columns if ' ' in c]

remove_cols = genres.tolist() + duplicated_cols + ['tconst', 'title']

Y = movies.Crime.values
X = movies[[c for c in movies.columns if c not in remove_cols]]

predictors = X.columns
X = X.values

print Y.shape
print X.shape

(7755,)
(7755, 201)


#### 3. Build and cross-validate your decision tree classifier

In [54]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import cross_val_score

In [64]:
dtc = DecisionTreeClassifier(max_depth = 6)

dtc_scores = cross_val_score(dtc, X, Y, cv= 5)
print dtc_scores
print np.mean(dtc_scores)

[ 0.69458763  0.68878866  0.69374597  0.70322581  0.70129032]
0.696327677607


In [65]:
baseline_acc = np.mean(Y)
print baseline_acc

dtc.fit(X, Y)

0.292327530625


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [66]:
feat_imp = dtc.feature_importances_
feature_importances = pd.DataFrame({'feat_imp': feat_imp, 'predictors': predictors})
feature_importances.sort(['feat_imp'], ascending = False, inplace = True)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


In [68]:
feature_importances.head(5)

,feat_imp,predictors
53,0.156024,family
111,0.132979,michael
164,0.109706,star wars
200,0.080507,user_rating
17,0.054559,book


#### 4. Gridsearch optimal parameters for your classifier. Does the performance improve?

In [71]:
from sklearn.grid_search import GridSearchCV

Y = movies.Action.values

dtc_gs_params = {
    'max_depth': [1, 2, 3, 4, 5, 6],
    'min_samples_split': [1, 25, 50, 100],
}

dtc_gs = GridSearchCV(dtc, dtc_gs_params, cv = 5)
dtc_gs.fit(X, Y)

GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'min_samples_split': [1, 25, 50, 100], 'max_depth': [1, 2, 3, 4, 5, 6]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [72]:
print dtc_gs.best_score_
print dtc_gs.best_params_
best_dtc = dtc_gs.best_estimator_

0.824371373308
{'min_samples_split': 1, 'max_depth': 2}


#### 5. Build and cross-validate your decision tree regressor

In [73]:
dtc_scores = cross_val_score(best_dtc, X, Y, cv=5)
print dtc_scores, np.mean(dtc_scores)

[ 0.78157216  0.83430045  0.83430045  0.83172147  0.84      ] 0.824378907522


In [76]:
from sklearn.tree import DecisionTreeRegressor

duplicated_cols = ['rating', 'year_x', 'num_votes', 'runtime']
ngram2_col = [c for c in movies.columns if ' ' in c]

remove_cols =  duplicated_cols + ['tconst', 'title'] + ngram2_col

Y = movies.user_rating.values

X = movies[[c for c in movies.columns if c not in remove_cols]]
 

dtr = DecisionTreeRegressor(max_depth = 2)

dtr_scores = cross_val_score(dtr, X, Y, cv = 5)
print dtr_scores, np.mean(dtr_scores)

dtr.fit(X, Y)

[ 0.96809731  0.96851651  0.96023477  0.9720083   0.96055384] 0.965882146779


DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

In [ ]:
feat_imp = dtc.feature_importances_
feature_importances = pd.DataFrame({'feat_imp': feat_imp, 'predictors': predictors})
feature_importances.sort(['feat_imp'], ascending = False, inplace = True)

#### 6. Gridsearch the optimal parameters for your classifier. Does performance improve?

In [ ]:
dtr_gs_params = {
    'max_depth': range(1, 20),
    'min_samples_split': range(1, 100, 10),
    'max_features':['auto', 'sqrt', 'log2']
}

dtr_gs = GridSearchCV(dtr, dtr_gs_params, cv = 5, verbose = 1)
dtr_gs.fit(X, Y)

---

## Part 6: Elastic Net


#### 1. Gridsearch optimal parameters for an ElasticNet using the regression target and predictors you used for the decision tree regressor.


In [ ]:
from sklearn.linear_model import ElasticNet

Xn = (X - X.mean()) / X.std()

enet = ElasticNet()

enet_params = {
    'l1_ratio': np.linspace(0.05, 1.0, 20),
    'alpha': np.logspace(-5, 1, 20)
}

enet_gs = GridSearchCV(enet, enet_params, cv = 5, verbose = 1)
enet_gs.fit(Xn, Y)

print enet_gs.best_params_
print enet_gs.best_score_

In [ ]:
enet = enet_gs.best_estimator_
enet.fit(Xn, Y)
coef = enet.coef_
enet_coefs = pd.DataFrame({'abs_coef': abs(coef), 'predictors': predictors})
enet_coefs.sort(['abs_coef'], ascending = False, inplace = True)
enet_coefs

#### 2. Is cross-validated performance better or worse than with the decision trees? 

#### 3. Explain why the elastic net may have performed best at that particular l1_ratio and alpha

---

## Part 7: Bagging and Boosting: Random Forests, Extra Trees, and AdaBoost

#### 1. Load the random forest regressor, extra trees regressor, and adaboost regressor from sklearn

#### 2. Gridsearch optimal parameters for the three different ensemble methods.

#### 3. Evaluate the performance of the two bagging and one boosting model. Which performs best?

#### 4. Extract the feature importances from the Random Forest regressor and make a DataFrame pairing variable names with their variable importances.

#### 5. Plot the ranked feature importances.

#### 6.1 [BONUS] Gridsearch an optimal Lasso model and use it for variable selection (make a new predictor matrix with only the variables not zeroed out by the Lasso). 

#### 6.2 [BONUS] Gridsearch your best performing bagging/boosting model from above with the features retained after the Lasso. Does the score improve?

#### 7.1. [BONUS] Select a threshold for variable importance from your Random Forest regressor and use that to perform feature selection, creating a new subset predictor matrix.

#### 7.2 [BONUS] Using BaggingRegressor with a base estimator of your choice, test a model using the feature-selected dataset you made in 7.1

---

## [VERY BONUS] Part 8: PCA

#### 1. Perform a PCA on your predictor matrix

#### 2. Examine the variance explained and determine what components you want to keep based on them.

#### 3. Plot the cumulative variance explained by the ordered principal components.

#### 4. Gridsearch an elastic net using the principal components you selected as your predictors. Does this perform better than the elastic net you fit earlier?

#### 5. Gridsearch a bagging ensemble estimator that you fit before, this time using the principal components as predictors. Does this perform better or worse than the original? 

#### 6. Look at the loadings of the original predictor columns on the first 3 principal components. Is there any kind of intuitive meaning here?

Hint, you will probably want to sort by absolute value of magnitude of loading, and also only look at the obviously important (larger) ones!

# [Extremely Bonus] Part 9:  Clustering

![](https://snag.gy/jPSZ6U.jpg)

 ***Bonus Bonus:***
This extended bonus question is asking to do something we never really talked about but would like for you to attempt based on the assumptions that we learned during this weeks clustering lesson(s).

#### 1. Import your favorite clustering module

#### 2. Encode categoricals

#### 3. Evaluate cluster metics solely based on a range of K
If K-Means:  SSE/Inertia vs Silhouette (ie: Elbow), silhouette average, etc

#### 4.  Look at your data based on the subset of your predicted clusters.
Assign the cluster predictions back to your dataframe in order to see them in context.  This is great to be able to group by cluster to get a sense of the data that clumped together.

#### 5. Describe your findings based on the predicted clusters 
_How well did it do?  What's good or bad?  How would you improve this? Does any of it make sense?_